## MLflow's Model Registry

In [42]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [43]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

AttributeError: 'MlflowClient' object has no attribute 'list_experiments'

In [ ]:
client.create_experiment(name="my-cool-experiment")

Let's check the latest versions for the experiment with id `1`...

In [44]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [45]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 167c33b63e114a498598380a64324db7, rmse: 6.2883
run id: 9d1d4cc86d1e4ce582e4729e20af4a19, rmse: 6.2970
run id: 3a229b5e4be942a0a3fa1c25a19292f3, rmse: 6.3018
run id: aa55945d42a0452487da861066929b10, rmse: 6.3024
run id: 0e33af07ebb5459cb33d5bd9c3238430, rmse: 6.3095


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [46]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [47]:
run_id = "167c33b63e114a498598380a64324db7"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/05/23 09:03:57 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 5
Created version '5' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1684832637601, current_stage='None', description=None, last_updated_timestamp=1684832637601, name='nyc-taxi-regressor', run_id='167c33b63e114a498598380a64324db7', run_link=None, source='/home/diego/mlops-zoomcamp/02-experiment-tracking/mlruns/1/167c33b63e114a498598380a64324db7/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

In [48]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: Staging
version: 5, stage: None


In [51]:
model_version = 3
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1684831279348, current_stage='Production', description='The model version 3 was transitioned to Staging on 2023-05-23', last_updated_timestamp=1684832660448, name='nyc-taxi-regressor', run_id='167c33b63e114a498598380a64324db7', run_link=None, source='/home/diego/mlops-zoomcamp/02-experiment-tracking/mlruns/1/167c33b63e114a498598380a64324db7/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [52]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1684831279348, current_stage='Production', description='The model version 3 was transitioned to Production on 2023-05-23', last_updated_timestamp=1684832664308, name='nyc-taxi-regressor', run_id='167c33b63e114a498598380a64324db7', run_link=None, source='/home/diego/mlops-zoomcamp/02-experiment-tracking/mlruns/1/167c33b63e114a498598380a64324db7/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [53]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [54]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [55]:
#run_id = "4f129340b595447fb2fb3acf475ec3d8"

client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

/tmp/ipykernel_37552/1526660350.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


MlflowException: The following failures occurred while downloading one or more artifacts from /home/diego/mlops-zoomcamp/02-experiment-tracking/mlruns/1/167c33b63e114a498598380a64324db7/artifacts: {'preprocessor': "FileNotFoundError(2, 'No such file or directory')"}

In [56]:
import pickle

with open("models/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [57]:
X_test = preprocess(df, dv)

In [58]:
target = "duration"
y_test = df[target].values

In [61]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

2023/05/23 09:05:49 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[09:05:49] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 30.9 s, sys: 18.5 ms, total: 30.9 s
Wall time: 7.9 s


{'rmse': 6.248294328516825}

In [62]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

2023/05/23 09:06:41 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[09:06:41] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 33.1 s, sys: 15.8 ms, total: 33.1 s
Wall time: 8.43 s


{'rmse': 6.248294328516825}

In [63]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1684832175246, current_stage='Production', description=None, last_updated_timestamp=1684832840578, name='nyc-taxi-regressor', run_id='167c33b63e114a498598380a64324db7', run_link=None, source='/home/diego/mlops-zoomcamp/02-experiment-tracking/mlruns/1/167c33b63e114a498598380a64324db7/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>